## Deliverable 3. Create a Travel Itinerary Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
#from config import g_key
import os
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("g_key")

# Configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
1,4,Shingu,JP,76.86,overcast clouds,33.7333,135.9833,Shingu UI Hotel
2,5,Jamestown,US,76.51,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,12,Hithadhoo,MV,81.57,heavy intensity rain,-0.6000,73.0833,Scoop Guest House
4,16,Sao Filipe,CV,77.65,overcast clouds,14.8961,-24.4956,Hotel Xaguate
5,18,San Policarpo,PH,80.42,light rain,12.1791,125.5072,Agapita M. Nicart
6,19,Douentza,ML,88.52,scattered clouds,14.9951,-2.9517,Ogomono Doumbo
7,23,Butaritari,KI,81.28,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
8,25,Tripoli,LY,83.52,clear sky,32.8752,13.1875,Al Waddan Hotel
9,27,Bolobo,CD,83.71,overcast clouds,-2.1667,16.2333,Hotel de Stephen


In [10]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [11]:
# 4a. Add a marker layer for each city to the map.
max_temp=vacation_df["Max Temp"]
fig=gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=max_temp,
                                 dissipating=False,
                              max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]=="Broome"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Broome"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Jamestown"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Westport"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Hamilton"]

In [17]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0],vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0],vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0],vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0],vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0],vacation_stop3["Lng"].to_numpy()[0]

In [18]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
#waypoints=[stop1,stop2,stop3]
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
direction=gmaps.directions_layer(start=start,end=end,
                                 waypoints=[stop1,stop2,stop3],
                                 travel_mode="DRIVING")
fig.add_layer(direction)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,269,Broome,US,82.02,broken clouds,42.2506,-75.8330,Chenango Valley State Park
1,5,Jamestown,US,76.51,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,272,Westport,US,88.29,broken clouds,41.1415,-73.3579,The Westport Inn
3,418,Hamilton,US,84.88,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast


In [22]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map.
fig=gmaps.figure()
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))